## Ce notebook sert à load les cons fichier pour tous mes autres notebooks dans le folder, comme ca j'ai pas à copier coller toutes ces étapes au début de chaque notebook :)

### Initialisation of the absolute Github repository path :

In [2]:
from pathlib import Path
import sys

try:
    root = Path(__file__).resolve().parent
except NameError:
    root = Path.cwd()  # fallback for Jupyter notebooks

while root.parent != root:
    if any((root / marker).exists() for marker in [".git", "README.md", "results.ipynb", "README.txt"]):
        break
    root = root.parent

# Fallback in case nothing found
if not any((root / marker).exists() for marker in [".git", "README.md", "results.ipynb", "README.txt"]):
    print("Could not locate project root — defaulting to current working directory")
    root = Path.cwd()

print(f"Root folder detected at: {root}")

# Ensure importability of the project
if str(root) not in sys.path:
    sys.path.insert(0, str(root))



Root folder detected at: c:\Users\cyrie\OneDrive\Documents\EPFL\2025_MA_1\ADA\ada-2025-project-adacore42


In [3]:
from src.utils.paths import STORED_DATAPREP_PKL_PATH, STORED_PLOTSGUI_PKL_PATH, DATA_PREPARATION_PY_PATH, STORED_DATAPREPNOUNS_PKL_PATH, STORED_DATAPREPTOKENS_PKL_PATH, OCCUPATIONS_CSV_PATH
print(DATA_PREPARATION_PY_PATH.resolve()) #If works: ...../ADAcore42/src/data/DataPreparation.py

C:\Users\cyrie\OneDrive\Documents\EPFL\2025_MA_1\ADA\ada-2025-project-adacore42\_Other\cycy_analysis\src\data\DataPreparation.py


### Imports:

In [ ]:
# Install Sentence Transformer for the semantic embedding purpose (Sentence-BERT (SBERT) model is used) :
# pip install -U sentence-transformers


# Download NLTK resources if not already
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

In [4]:
# utils
from src.utils.general_utils import *

# paths
from src.utils.paths import *

# working librairies
import os
import pickle


# classic librairies
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

# text processing librairies
import nltk
from collections import Counter
import re
from textblob import TextBlob
from sentence_transformers import SentenceTransformer, SimilarityFunction, util
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# statistical libraries
from statsmodels.stats import diagnostic
from scipy.stats import spearmanr, pearsonr
import networkx as nx

# clustering
from sklearn.cluster import KMeans
import umap


c:\Users\cyrie\anaconda3\envs\ada\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading of preprocessed data's pickle :

In [5]:
stored_dataprep_pkl_path = root / STORED_DATAPREP_PKL_PATH

# Chargement du fichier pickle
with open(stored_dataprep_pkl_path, "rb") as f:
    data = pickle.load(f)

# Extraction des objets stockés dans le pickle

# dataA est une liste de DataFrames pandas (ou un objet similaire, comme un dictionnaire de DataFrames). Chaque élément de la liste contient un DataFrame avec 7 colonnes et un nombre variable de lignes.
dataA = data['dataA']
# dataC est un DataFrame de métadonnées de tous les cartoon contests.
dataC = data['dataC']
dataA_startID = data['dataA_startID']
dataA_endID = data['dataA_endID']
dataC_lastGoodID = data['dataC_lastGoodID']


In [6]:
dataA[77]

,caption,mean,precision,votes,not_funny,somewhat_funny,funny
rank,,,,,,,
0,Can I tell you about a few items that aren't o...,1.962439,0.015871,2476,820,929,727
1,We're out of that too.,1.960586,0.018005,1979,675,707,597
2,Thank you for actually reading the entire menu...,1.929282,0.018846,1810,650,638,522
3,"If you'd like, I can have the sommelier drag o...",1.913781,0.020508,1415,487,563,365
4,Wait 'til you see the check.,1.913360,0.015410,2747,1025,935,787
...,...,...,...,...,...,...,...
6085,You're the smallest person we've ever had here!,1.000000,0.016261,62,62,0,0
6086,Wait till you see the size of our dessert menu!,1.000000,0.016261,62,62,0,0
6087,Welcome to Texas.,1.000000,0.005168,194,194,0,0


In [7]:
dataC.head()

,num_captions,num_votes,image_locations,image_descriptions,image_uncanny_descriptions,entities,questions,date
0,3905.0,41185.0,[the street],[A man is relaxing on a city street. Others ar...,[A man is just laying in the middle of the sid...,[https://en.wikipedia.org/wiki/Bystander_effec...,[Why is he laying there?],NaT
1,3325.0,28205.0,"[the front hard, a residential walkway]",[A man in a winter coat and cap is looking at ...,[It's unusual to see someone holding a snow sh...,"[https://en.wikipedia.org/wiki/Snowball_fight,...",[Is the man overly small or the shovel overly ...,NaT
2,4399.0,21574.0,"[yoga place, a yoga studio]",[A man and woman are standing facing one anoth...,[Nothing is really out of place in this image....,"[https://en.wikipedia.org/wiki/Rug, https://en...","[Why is the man carrying a huge rug?, Why is t...",2016-03-21
3,4141.0,16894.0,"[a workplace, an elevator]",[Three business men are walking down a hall. T...,[A suit case is usually carried by one person ...,[https://en.wikipedia.org/wiki/Worker_cooperat...,[Why is the briefcase big enough for three peo...,2016-03-27
4,3951.0,95790.0,[plains],[Some cowboys are riding through the desert. T...,[There are rocking horses in place of real hor...,"[https://en.wikipedia.org/wiki/Rocking_horse, ...",[Why is this chase taking place?],2016-04-03
